In [8]:
pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5955 sha256=05e4664fdf0a7cb4c9abda34937d6eb95299ee1be49ec455ad7f29bc847b9427
  Stored in directory: /Users/milindbhatia/Library/Caches/pip/wheels/84/45/a4/8ade6576f75410d8162c6da1de0aa9df56c16c711acab5a813
Successfully built apyori
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [33]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import plotly.express as px

/Users/milindbhatia/anaconda3/lib/python3.10/site-packages/plotly/express/imshow_utils.py:24: DeprecationWarning:

`np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)



In [5]:
#Reading the file

file_p = '/Users/milindbhatia/Desktop/Study/Summer/Intro to ML/Take Home - Group Assignment/Intro-to-ML-project/Data/groceries.txt'
data_items = []

with open(file_p,'r') as f:
    for line in f:
        items = line.strip().split(',')
        data_items.append(items)

In [24]:
unique_items = set(item for sublist in data_items for item in sublist)

# Initialize an empty dictionary to store the data
data_dict = {item: [] for item in unique_items}

# Populate the dictionary with True/False values
for item in unique_items:
    for sublist in data_items:
        data_dict[item].append(item in sublist)

# Create a DataFrame from the dictionary
df = pd.DataFrame(data_dict)

      abrasive cleaner  pot plants  flour  hamburger meat  cooking chocolate  \
0                False       False  False           False              False   
1                False       False  False           False              False   
2                False       False  False           False              False   
3                False       False  False           False              False   
4                False       False  False           False              False   
...                ...         ...    ...             ...                ...   
9830             False       False   True            True              False   
9831             False       False  False           False               True   
9832             False       False  False           False              False   
9833             False       False  False           False              False   
9834             False       False  False           False              False   

       soap  finished products  dessert

In [31]:
all_items = [item for sublist in data_items for item in sublist]

item_counts = pd.Series(all_items).value_counts()

freq_df = pd.DataFrame({'Item': item_counts.index, 'Frequency': item_counts.values})

freq_df.head(20)

,Item,Frequency
0,whole milk,2513
1,other vegetables,1903
2,rolls/buns,1809
3,soda,1715
4,yogurt,1372
5,bottled water,1087
6,root vegetables,1072
7,tropical fruit,1032
8,shopping bags,969
9,sausage,924


In [35]:
freq_df30 = freq_df.head(30)
fig = px.treemap(freq_df30, path=['Item'], values='Frequency', color='Frequency',
                 color_continuous_scale='viridis', title='Item Frequency Treemap',
                 hover_data=['Frequency']) 

fig.show()

/Users/milindbhatia/anaconda3/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/Users/milindbhatia/anaconda3/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/Users/milindbhatia/anaconda3/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/Users/milindbhatia/anaconda3/lib/python3.10/site

In [47]:
#Applying Apriori Algorithm on all items

frq_items = apriori(df, min_support = 0.03, use_colnames=True)
print(frq_items.sort_values(by='itemsets',ascending=False))

rules = association_rules(frq_items, metric="lift", min_threshold=1.5)
rules.sort_values(by='lift',ascending=False)

     support                             itemsets
0   0.033249                     (hamburger meat)
47  0.056024                 (yogurt, whole milk)
34  0.033452                           (UHT-milk)
35  0.033859                              (sugar)
36  0.057651                               (pork)
..       ...                                  ...
26  0.055414                             (butter)
27  0.053279                               (curd)
28  0.038434                            (waffles)
29  0.033249                            (berries)
62  0.047382  (root vegetables, other vegetables)

[63 rows x 2 columns]


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
19,(other vegetables),(root vegetables),0.193493,0.108998,0.047382,0.244877,2.246605,0.026291,1.179941,0.688008
18,(root vegetables),(other vegetables),0.108998,0.193493,0.047382,0.434701,2.246605,0.026291,1.426693,0.622764
8,(sausage),(rolls/buns),0.093950,0.183935,0.030605,0.325758,1.771048,0.013324,1.210344,0.480506
9,(rolls/buns),(sausage),0.183935,0.093950,0.030605,0.166390,1.771048,0.013324,1.086899,0.533490
2,(tropical fruit),(other vegetables),0.104931,0.193493,0.035892,0.342054,1.767790,0.015589,1.225796,0.485239
3,(other vegetables),(tropical fruit),0.193493,0.104931,0.035892,0.185497,1.767790,0.015589,1.098913,0.538522
13,(whipped/sour cream),(whole milk),0.071683,0.255516,0.032232,0.449645,1.759754,0.013916,1.352735,0.465077
12,(whole milk),(whipped/sour cream),0.255516,0.071683,0.032232,0.126144,1.759754,0.013916,1.062323,0.579917
14,(whole milk),(root vegetables),0.255516,0.108998,0.048907,0.191405,1.756031,0.021056,1.101913,0.578298
15,(root vegetables),(whole milk),0.108998,0.255516,0.048907,0.448694,1.756031,0.021056,1.350401,0.483202
